# Подготовка к написанию запросов

In [84]:
import warnings
import pandas as pd
import psycopg2

### Чтение данных для соединения 

In [85]:
# Чтение данных из файла
with open('connection.txt', 'r') as f:
    lines = f.read().splitlines()

connection_params = {}
# Каждая строка заполняется на примере след. паттерна:
# PORT = 8000
for line in lines:
    key, value = line.split(' = ', 1)
    connection_params[key.lower()] = value

### Уставновка соединений с базой данных

In [86]:
engine = psycopg2.connect(
   dbname=connection_params['dbname'],
   user=connection_params['user'],
   host=connection_params['host'],
   password=connection_params['password'],
   port=connection_params['port']
)

# Отключение сообщений
warnings.filterwarnings("ignore", category=UserWarning)

 # Предварительный анализ данных

### Количество вакансий в базе

In [87]:
query = f'''
        select
            count(*)
        from
            vacancies
        '''
amount_vac = pd.read_sql_query(query, engine)
print(f'Количество вакансий в БД = {amount_vac['count'].iloc[0]}')

Количество вакансий в БД = 49197


### Количество работодателей

In [88]:
query = f'''
        select
            count(*) amount_emp
        from
            employers
        '''
amount_emp = pd.read_sql_query(query, engine)
print(f'{amount_emp['amount_emp'].iloc[0]} работодателей представлены в БД')

23501 работодателей представлены в БД


### Количество регионов

In [89]:
query = f'''
        select
            count(*) amount_areas
        from
            areas
        '''
amount_areas = pd.read_sql_query(query, engine)
print(f'{amount_areas['amount_areas'].iloc[0]} региона в БД')

1362 региона в БД


### Количество сфер деятельности

In [90]:
query = f'''
        select
            count(*) amount_industries
        from
            industries
        '''
amount_industries = pd.read_sql_query(query, engine)
print(f'{amount_industries['amount_industries'].iloc[0]} сферы деятельности')

294 сферы деятельности


# Детальный анализ вакансий

Пятерка лидеров по количеству вакансий

In [91]:
query = f'''
        select
            a.name,
            count(v.id) amount_vac
        from
            vacancies v
            left join areas a on a.id = v.area_id
        group by a.id
        order by count(v.id) desc
        limit 5
        '''
top_regions_by_vac = pd.read_sql_query(query, engine)
top_regions_by_vac

,name,amount_vac
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


In [92]:
query = f'''
        -- Создание временной таблицы для будущего объединения
        with t as(
            select
                a.name,
                count(v.id) amount_vac
            from
                vacancies v
                left join areas a on a.id = v.area_id
            group by a.id
            order by count(v.id) desc
            limit 5
        )
        select
            *
        from
            t
        -- Подсчет суммы вакансий и объединение
        union all
        select
            'Total',
            sum(amount_vac)
        from t
        '''
top_regions_by_vac_sum = pd.read_sql_query(query, engine)
top_regions_by_vac_sum

,name,amount_vac
0,Москва,5333.0
1,Санкт-Петербург,2851.0
2,Минск,2112.0
3,Новосибирск,2006.0
4,Алматы,1892.0
5,Total,14194.0


Большинство вакансий отноятся к столицам (Новосибирск - столица Сибирского федерального округа).  
А точнее ~1/3 всех вакансий приходится на эти города.  
Следовательно, можно сделать вывод, что большинство вакансий формируются в крупных городах.  
Этим объясняется желаение многих людей переезжать в данные регионы.

Количество резюме, где заполнено поле хотя бы с одной зарплатой

In [93]:
query = f'''
        select
            count(*) cnt
        from
            vacancies
        where
            (salary_from is not null) or (salary_to is not null)
        '''
amount_vac_with_sal = pd.read_sql_query(query, engine)
print(f'У {amount_vac_with_sal['cnt'].iloc[0]} вакансий заполнено хотя бы одно поле с зарплатой')

У 24073 вакансий заполнено хотя бы одно поле с зарплатой


Многие(половина) не хотят указывать уровень ЗП по некоторым возможным причинам:
* Желание обсудить ЗП на собеседовании, чтобы "не продешевить";
* Непонимание текущих ЗП, какие реалии на рынке;
* Готовность к рассмотрению разных предложений, вне зависимости от уровня ЗП
* и тд

### Среднее значение для нижней и верхней границы зарплатной вилки

In [94]:
query = f'''
        select
            round(avg(salary_from)) avg_salary_from,
            round(avg(salary_to)) avg_salary_to
        from
            vacancies
        '''
avg_salary = pd.read_sql_query(query, engine)
print(f'Средняя зарплата по нижней границе = {avg_salary['avg_salary_from'].iloc[0]}')
print(f'Средняя зарплата по верхней границе = {avg_salary['avg_salary_to'].iloc[0]}')


Средняя зарплата по нижней границе = 71065.0
Средняя зарплата по верхней границе = 110537.0


In [95]:
query = f'''
        select
            experience,
            round(avg(salary_from)) avg_salary_from,
            round(avg(salary_to)) avg_salary_to
        from
            vacancies
        group by
            experience
        order by
            round(avg(salary_from)) desc,
            round(avg(salary_to)) desc
        '''
avg_salary_by_exp = pd.read_sql_query(query, engine)
avg_salary_by_exp


,experience,avg_salary_from,avg_salary_to
0,Более 6 лет,150156.0,197819.0
1,От 3 до 6 лет,112544.0,171974.0
2,От 1 года до 3 лет,64456.0,97436.0
3,Нет опыта,34699.0,56990.0


Показатель по общей средней ЗП неинформативен, так видно, что ЗП напрямую зависит от опыта работы.  
То есть для стажера, который ищет работу, вилка - 71065 - 110537 будет неатуальна.

### Количество вакансий для каждого сочетания: рабочего графика + типа трудоустройства

In [96]:
query = f'''
        select
            schedule,
            employment,
            count(id) amount
        from
            vacancies
        group by
            schedule,
            employment
        order by count(id) desc
        '''
sch_emp = pd.read_sql_query(query, engine)
display(sch_emp)


,schedule,employment,amount
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


Наблюдается зависимость зависимости количество вакансий от вида трудоустройства, тогда как от типа работчего графика - не так.  
Большинству требуется "Полная занятость", так как:
* В основном такие сотрудники лучше соориентированы в делах компании
* Многие задачи требуют постоянного присутствия работников на рабочем месте, что упрощает коммуникацию
* и тд

### Количество вакансий по требуемому опыту

In [97]:
query = f'''
        select
            experience,
            count(*) amount_vac
        from
            vacancies
        group by
            experience
        order by count(*)
        '''
exp_vac = pd.read_sql_query(query, engine)
display(exp_vac)


,experience,amount_vac
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


Можно сделать выводы:
* Самый востребованный опыт работы - 1-3 года, так как работник с таким опытом уже понимает, как работает крупная компания, есть понимание работы в команде, также определенный опыт в работе с реальными объектами (людьми). При этом уровень средней ЗП не так высок, следовательно, это самые "ходовые" сотрудники.
* При этом в основном поиски работы ведуться с видом трудоустройства - Полная занятость.
* Уровень средней ЗП в напрямую зависит от опыта работы.

# Анализ работодателей

### Распредение работодателей по количеству вакансий

In [98]:
query = f'''
        select
            e.name,
            count(v.id) amount
        from
            vacancies v
            left join employers e on v.employer_id = e.id
        group by
            e.id
        order by
            count(v.id) desc
        limit 5
        '''
amount_vac_by_emp = pd.read_sql_query(query, engine)
display(amount_vac_by_emp)

,name,amount
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


Как и ожидалось, в основном самые крупные игроки на рынке ищут большее количество людей.

### Среди регионов, в которых нет вакансий, поиск того, в котором наибольшее количество работодателей

In [99]:
query = f'''
        select
            a.name,
            count(e.id) cnt
        from
            areas a
            left join vacancies v on v.area_id = a.id
            join employers e ON e.area = a.id
        where v.id is null
        group by a.name
        order by cnt desc
        limit 1
        '''
res_reg = pd.read_sql_query(query, engine)
print(f'{res_reg['name'].iloc[0]} - это регион, в котором нет вакансий и наибольшее кол-во работодатей')

Россия - это регион, в котором нет вакансий и наибольшее кол-во работодатей


Скорее всего ошибка заполнения анкеты, когда в качестве региона-работодателя указывается его "глобальное" местопложение, а ищут работы по регионам, указывая их в резюме.

### Подсчет количества регионов Для каждого работодателя, в которых он публикует свои вакансии.  
Выберите максимальное значение из получившегося списка


In [100]:
query = f'''
        select
            e.name,
            count(distinct v.area_id) cnt
        from
            employers e
            left join areas a on e.area = a.id
            left join vacancies v on e.id = v.employer_id
        group by e.name
        order by cnt desc
        limit 5
        '''
        
emp_reg = pd.read_sql_query(query, engine)
print(f'Компания {emp_reg['name'].iloc[0]} публикует вакансии в {emp_reg['cnt'].iloc[0]} регионе')

Компания Яндекс публикует вакансии в 181 регионе


### Запрос для подсчёта количества работодателей, у которых не указана сфера деятельности.

In [101]:
query = f'''
        select
            count(*)
        from
            employers e
            left join employers_industries e_i on e.id = e_i.employer_id
        where
            e_i.industry_id is null
        '''
        
cnt_emp_without_industies = pd.read_sql_query(query, engine)
print(f'У {cnt_emp_without_industies['count'].iloc[0]} работодателей не указана сфера деятельности')

У 8419 работодателей не указана сфера деятельности


Данный показатель может быть связан с рядом причин:
* Неправильное оформление профиля компании (ошибка)
* Широкий спектр деятельности компании
* Либо компания ищет сотрудников через рекрутинговые агенства  
Это только ряд причин, могут быть и другие

### Название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности.

In [102]:
query = f'''
        select
            e.name
            --count(e_i.industry_id) cnt
        from
            employers e
            left join employers_industries e_i on e.id = e_i.employer_id
        group by
            e.id
        having 
            count(e_i.industry_id) = 4
        order by 
            e.name
        offset 2 limit 1
        '''
        
third_comp_with4_industries = pd.read_sql_query(query, engine)
print(f' Компания {third_comp_with4_industries['name'].iloc[0]} находится на 3 месте по алфавиту, у которой указано 4 типа деятельности')

 Компания 2ГИС находится на 3 месте по алфавиту, у которой указано 4 типа деятельности


### У какого количества работодателей в качестве сферы деятельности указана «Разработка программного обеспечения».

In [103]:
query = f'''
        select
            count(e_i.employer_id) cnt
        from
            employers_industries e_i
            join industries i on i.id = e_i.industry_id
        where i.name = 'Разработка программного обеспечения'
        '''
        
amount_emp = pd.read_sql_query(query, engine)
print(f'У {amount_emp['cnt'].iloc[0]} работодателей в качестве сферы деятельности указана "Разработка программного обеспечения"')

У 3553 работодателей в качестве сферы деятельности указана "Разработка программного обеспечения"


### Кол-во вакансий для компании "Яндекс" в городах-миллионниках

In [104]:
query = f'''
        -- Собираем данные о вакансиях в городах-миллионниках
        with region_vacancies as(
            select
                a.name area_name,
                v.employer_id
            from
                vacancies v
                join areas a on v.area_id = a.id
            where
                a.name in('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Красноярск', 'Нижний Новгород', 'Челябинск', 'Уфа', 'Самара', 'Ростов-на-Дону', 'Краснодар', 'Омск', 'Воронеж', 'Пермь', 'Волгоград')
        ),
        -- Выполняем групировку по регионам и считаем кол-во вакансий
        region_counts as(
            select
            r_v.area_name,
            count(*) cnt
            from
                region_vacancies r_v
                join employers e on e.id = r_v.employer_id
            where
                e.name = 'Яндекс'
            group by
                r_v.area_name
        )
        select
            area_name,
            cnt
        from region_counts
        -- Подсчет и объединение результатов
        union all
        select
            'Total',
            sum(cnt)
        from region_counts
        order by cnt
        '''
        
yandex_regions = pd.read_sql_query(query, engine)
yandex_regions

,area_name,cnt
0,Омск,21.0
1,Челябинск,22.0
2,Красноярск,23.0
3,Волгоград,24.0
4,Пермь,25.0
5,Казань,25.0
6,Ростов-на-Дону,25.0
7,Самара,26.0
8,Уфа,26.0
9,Краснодар,30.0


Компания "Яндекс" - это крупнейшая компания, у которой Самое больше количество вакансий, регионов, в которых они нанимают сотрудников.

# Предметный анализ

Сколько вакансий имеет отношение к данным

In [105]:
query = '''
    select
        count(distinct id) cnt
    from
        vacancies
    where
        -- Поиск вакансии по заданному паттерну
        lower(name) like any(array['%data%', '%данн%'])
'''

# Выполнение запроса
data_vac = pd.read_sql_query(query, engine)

# Правильный способ использования переменной в функции print
print(f"{data_vac['cnt'].iloc[0]} вакансий имеет отношение к данным")

1771 вакансий имеет отношение к данным


In [106]:
query = '''
    select
        count(distinct id) cnt
    from
        vacancies
    where
        -- Поиск вакансии по заданному паттерну
        (lower(name) like any(array['%data scientist%', '%data science%', '%исследователь данных%', '%machine learning%', '%машинн%обучен%'])
        or (name like '%ML%' and name not like '%HTML%' ))
'''

# Выполнение запроса
ds_vac = pd.read_sql_query(query, engine)

# Правильный способ использования переменной в функции print
print(f"{ds_vac['cnt'].iloc[0]} вакансий имеет отношение к данным")

480 вакансий имеет отношение к данным


Всего 1/4 от общего количества вакансий, относящихся к данным, связаны с DS.  


### Количество подходящих вакансий для начинающего дата-сайентиста

In [107]:
query = f'''
        select
            count(*) cnt
        from
            vacancies
        where
            -- Поиск одного из подходящих сочетаний для дата-сайентиста (без HTML)
            (lower(name) like any(array['%data scientist%', '%data science%', '%исследователь данных%', '%machine learning%', '%машинн%обучен%'])
            or (name like '%ML%' and name not like '%HTML%' ))
            -- Отбор вакансий Junior
            and ((lower(name) like '%junior%') or (experience = 'Нет опыта') or employment = 'Стажировка')
        '''
        
jun_ds = pd.read_sql_query(query, engine)
print(f'Для начинающего дата-сайентиста подходит всего {jun_ds['cnt'].iloc[0]} вакансия')

Для начинающего дата-сайентиста подходит всего 51 вакансия


Всего 10% от общего количества вакансий DS относятся к Junior сотрудникам.  
Мало работодателей хотят тратить свое время и усилия на неопытных сотрудников, так как это может неокупиться, то есть есть вероятность, что сотрудник по истечении испытательного срока не подойдет.

### Количество вакансий для DS, с ключевыми навыками SQL или Posgres

In [108]:
query = f'''
        select
            count(*) cnt
        from
            vacancies
        where
            -- Поиск одного из подходящих сочетаний для дата-сайентиста (без HTML)
            (lower(name) like any(array['%data scientist%', '%data science%', '%исследователь данных%', '%machine learning%', '%машинн%обучен%'])
            or (name like '%ML%' and name not like '%HTML%' ))
            -- Ключевые навыки = SQL или Posgres
            and (lower(key_skills) like any(array['%sql%', '%postgres%']))
        '''
        
sql_ds = pd.read_sql_query(query, engine)
print(f'{sql_ds['cnt'].iloc[0]} Вакансий для DS, с ключевыми навыками SQL или Posgres')

201 Вакансий для DS, с ключевыми навыками SQL или Posgres


### Насколько популярен Python в вакансиях с DS

In [109]:
query = f'''
        select
            count(*) cnt
        from
            vacancies
        where
            -- Поиск одного из подходящих сочетаний для дата-сайентиста (без HTML)
            (lower(name) like any(array['%data scientist%', '%data science%', '%исследователь данных%', '%machine learning%', '%машинн%обучен%'])
            or (name like '%ML%' and name not like '%HTML%' ))
            -- Ключевые навыки заполнены
            and lower(key_skills) like '%python%' 
        '''
sql_ds = pd.read_sql_query(query, engine)
print(f'{sql_ds['cnt'].iloc[0]} Вакансии для DS, с требованием знания Python')

351 Вакансии для DS, с требованием знания Python


Половина вакансий требуется со знанием SQL, но практически все - со знанием Python.  
Потому что Python - это один из самых распространенных языков для DS.

###  Среднее количество ключевых навыков, указываемых для вакансий с DS

In [110]:
query = f'''
        select
            count(key_skills) skill_count,
            -- Вычитается изначальная длина строки и с удаленными символами табуляции
            -- Прибавляется 1, чтобы учесть последний ключевой навык
            -- Получается кол-во ключевых навыков, так как каждый отделен символом табуляции
            round(avg(length(key_skills) - length(replace(key_skills, chr(9), '')) + 1), 2) avg_skills
        from
            vacancies
        where
            -- Поиск одного из подходящих сочетаний для дата-сайентиста (без HTML)
            (lower(name) like any(array['%data scientist%', '%data science%', '%исследователь данных%', '%machine learning%', '%машинн%обучен%'])
            or (name like '%ML%' and name not like '%HTML%' ))
            -- Ключевые навыки заполнены
            and key_skills is not null
        '''
        
key_skills_ds = pd.read_sql_query(query, engine)
print(f'Среднее число указываемых навыков = {key_skills_ds['avg_skills'].iloc[0]}')

Среднее число указываемых навыков = 6.41


In [111]:
query = f'''
        select
            count(key_skills) skill_count,
            -- Вычитается изначальная длина строки и с удаленными символами табуляции
            -- Прибавляется 1, чтобы учесть последний ключевой навык
            -- Получается кол-во ключевых навыков, так как каждый отделен символом табуляции
            round(avg(length(key_skills) - length(replace(key_skills, chr(9), '')) + 1), 2) avg_skills
        from
            vacancies
        where
            -- Ключевые навыки заполнены    
            not (
                lower(name) like any(array['%data scientist%', '%data science%', '%исследователь данных%', '%machine learning%', '%машинн%обучен%'])
                or (name like '%ML%' and name not like '%HTML%')
            )
            and key_skills is not null
        '''
        
key_skills_general = pd.read_sql_query(query, engine)
print(f'Среднее число указываемых навыков = {key_skills_general['avg_skills'].iloc[0]}')

Среднее число указываемых навыков = 6.37


В сравнении можно заметить, что разницы в среднем количестве указываемых ключевых навыков - нет.  
Так что это неинформативный показатель, отличаются виды ключевых навыков, но не их количество

In [112]:
query = f'''
        select
            experience,
            -- Расчет средней ЗП
            round(avg(case
                when (salary_from is not null) and (salary_to is not null) then (salary_to + salary_from ) / 2
                when salary_from is not null then salary_from
                else salary_to
            end)) avg_salary
        from
            vacancies
        where
            -- Поиск одного из подходящих сочетаний для дата-сайентиста (без HTML)
            (lower(name) like any(array['%data scientist%', '%data science%', '%исследователь данных%', '%machine learning%', '%машинн%обучен%'])
            or (name like '%ML%' and name not like '%HTML%'))
            -- Хотя бы одно из полей ЗП заполнено
            and ((salary_from is not null) or (salary_to is not null) )
        group by experience
        '''
        
avg_salary_ds = pd.read_sql_query(query, engine)
avg_salary_ds

,experience,avg_salary
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


In [113]:
query = f'''
        select
            experience,
            -- Расчет средней ЗП
            round(avg(case
                when (salary_from is not null) and (salary_to is not null) then (salary_to + salary_from ) / 2
                when salary_from is not null then salary_from
                else salary_to
            end)) avg_salary
        from
            vacancies
        where
            -- Хотя бы одно из полей ЗП заполнено
            (salary_from is not null) or (salary_to is not null)
        group by experience
        order by avg_salary 
        '''
        
avg_salary_general = pd.read_sql_query(query, engine)
avg_salary_general

,experience,avg_salary
0,Нет опыта,40472.0
1,От 1 года до 3 лет,76541.0
2,От 3 до 6 лет,133211.0
3,Более 6 лет,166256.0


Как видно, средний уровень ЗП у DS в 2 раза больше, вне зависимтости от опыта работы.  
Это может быть связано с требованиями, необходимыми для работы в этом направлении, сложности професии и достаточно глубоких знаний в математике.

# Окончание основной части

### Исследование по количеству вакансий по городам и сферам деятельности

In [114]:
query = f'''
        select
            a.name area_name,
            i.name ind_name
        from
            vacancies v
            left join areas a on v.area_id = a.id
            left join employers e on v.employer_id = e.id
            left join employers_industries e_i on v.employer_id = e_i.employer_id
            join industries i on e_i.industry_id = i.id
        group by a.id, i.id
        order by count(*) desc
        limit 40
        '''
        
area_industry = pd.read_sql_query(query, engine)
area_industry

,area_name,ind_name
0,Москва,Разработка программного обеспечения
1,Москва,"Системная интеграция, автоматизации технологи..."
2,Минск,Разработка программного обеспечения
3,Санкт-Петербург,Разработка программного обеспечения
4,Москва,"Интернет-компания (поисковики, платежные систе..."
5,Санкт-Петербург,"Системная интеграция, автоматизации технологи..."
6,Москва,Банк
7,Новосибирск,Разработка программного обеспечения
8,Алматы,"Системная интеграция, автоматизации технологи..."
9,Нижний Новгород,Разработка программного обеспечения


90% самых популярных вакансий - это связанные с IT и в городах-миллиониках.  
В последние годы направление IT сильно развивается и все больше людей хотят работать именно там.

### Распределение средних заработных плат DS по регионам и опыту работы

In [115]:
query = f'''
        select
            v.experience,
            a.name area_name,
            -- Расчет средней ЗП
            round(avg(case
                when (salary_from is not null) and (salary_to is not null) then (salary_to + salary_from ) / 2
                when salary_from is not null then salary_from
                else salary_to
            end)) avg_salary
        from
            vacancies v
            left join areas a on v.area_id = a.id
        where
            -- Поиск одного из подходящих сочетаний для дата-сайентиста (без HTML)
            (lower(v.name) like any(array['%data scientist%', '%data science%', '%исследователь данных%', '%machine learning%', '%машинн%обучен%'])
            or (v.name like '%ML%' and v.name not like '%HTML%'))
            -- Хотя бы одно из полей ЗП заполнено
            and ((salary_from is not null) or (salary_to is not null))
        group by 
            v.experience,
            a.id
        order by avg_salary desc
        limit 15
        '''
        
avg_sal_by_exp_area = pd.read_sql_query(query, engine)
avg_sal_by_exp_area

,experience,area_name,avg_salary
0,От 3 до 6 лет,Кипр,300000.0
1,От 3 до 6 лет,Санкт-Петербург,300000.0
2,От 3 до 6 лет,Армения,268863.0
3,От 3 до 6 лет,Москва,263750.0
4,От 3 до 6 лет,Сербия,233794.0
5,От 3 до 6 лет,Черногория,233794.0
6,От 3 до 6 лет,Турция,233794.0
7,От 3 до 6 лет,Новосибирск,209500.0
8,От 3 до 6 лет,Белгород,200000.0
9,От 1 года до 3 лет,Санкт-Петербург,175173.0


Уровень средней заработной платы в основном зависит от опыта работы, как было сказано ранее.  
Но также видно, что регион вакансии также имеет влияние на ЗП, правда меньшее.

Связана данная зависимость скорее всего с тем, что в данное время люди с опытом могут переехать в другой город, на лучшее предложение, а не соглашаться на меньшее в своем регионе.  
Поэтому работодатели стараются держать ЗП +- на одном уровне.

## Закрытие соединения

In [116]:
engine.close()